In [35]:
import os
import io
# import ipynb.fs.defs.otzsu_komplett

In [25]:
#auf Bilder anwenden

#N2DH-GOWT1
N2DH_img_path = '../BilderDaten/BilderDaten/N2DH-GOWT1/img/'
N2DH_img_files = os.listdir(N2DH_img_path)
N2DH_result_path = '../Ergebnisse/Otsu Ergebnisse/N2DH-GOWT1/'
for file in N2DH_img_files:
    file_path = os.path.join(N2DH_img_path,file)
    result_file_path = os.path.join(N2DH_result_path,file)
    result = otsu_thresholding(io.imread(file_path))
    io.imsave(result_file_path,result)
    
#N2DL-HeLa

NameError: name 'otsu_thresholding' is not defined